__Шаг 1. Откройте таблицу и изучите общую информацию о данных__


Импорт необходимых библиотек:

In [61]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StringType

In [62]:
spark = SparkSession.builder.appName("data").getOrCreate()
df = spark.read.option("header", "true").csv("data.csv")

df.printSchema()
df.show()

root
 |-- children: string (nullable = true)
 |-- days_employed: string (nullable = true)
 |-- dob_years: string (nullable = true)
 |-- education: string (nullable = true)
 |-- education_id: string (nullable = true)
 |-- family_status: string (nullable = true)
 |-- family_status_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- income_type: string (nullable = true)
 |-- debt: string (nullable = true)
 |-- total_income: string (nullable = true)
 |-- purpose: string (nullable = true)

+--------+-------------------+---------+-------------------+------------+----------------+----------------+------+-----------+----+------------------+--------------------+
|children|      days_employed|dob_years|          education|education_id|   family_status|family_status_id|gender|income_type|debt|      total_income|             purpose|
+--------+-------------------+---------+-------------------+------------+----------------+----------------+------+-----------+----+---------------

__Шаг 2. Предобработка данных__

В двух столбцах есть пропущенные значения. Один из них — days_employed. Пропуски в этом столбце вы обработаете на следующем этапе. Найдите другой столбец и заполните пропущенные значения в нём медианным значением:
опишите, какие пропущенные значения вы обнаружили;
- проверьте, какую долю составляют пропущенные значения в каждом из столбцов с пропусками;
- приведите возможные причины появления пропусков в данных;
- объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

In [63]:
print("Столбцы, имеющие пропущенные (Null) значения:")
for column in df.columns:
    if df.filter(col(column).isNull()).count() > 0:
        print(f" - {column}")


Столбцы, имеющие пропущенные (Null) значения:
 - days_employed
 - total_income


Пропущенные значения есть в days_employed и total_income

In [64]:
cnt_row = df.count()
null_days_employed = df.where(col("days_employed").isNull()).count()
null_total_income = df.where(col("total_income").isNull()).count()

print('Доли пропусков в столбцах ДатаФрейма:')
print(f"- days_employed: {null_days_employed/cnt_row*100}%")
print(f"- total_income: {null_days_employed/cnt_row*100}%")

Доли пропусков в столбцах ДатаФрейма:
- days_employed: 10.099883855981417%
- total_income: 10.099883855981417%


Доля пропусков равна 10.10%, пропуски в одинаковых строках

Причины возникновения: 
1. Клиенты не предоставили информацию для данных
2. Баги сервиса, который принимал информацию
3. Нет точной информации

Мы заполним пропуски в total_income медианным значением, потому что этот метод более устойчив к выбросам и непредсказуемым значениям

In [65]:
df = df.fillna({"total_income": df.select(median("total_income")).first()[0]})
df.show()

+--------+-------------------+---------+-------------------+------------+----------------+----------------+------+-----------+----+------------------+--------------------+
|children|      days_employed|dob_years|          education|education_id|   family_status|family_status_id|gender|income_type|debt|      total_income|             purpose|
+--------+-------------------+---------+-------------------+------------+----------------+----------------+------+-----------+----+------------------+--------------------+
|       1| -8437.673027760233|       42|             высшее|           0| женат / замужем|               0|     F|  сотрудник|   0| 253875.6394525987|       покупка жилья|
|       1| -4024.803753850451|       36|            среднее|           1| женат / замужем|               0|     F|  сотрудник|   0|112080.01410244203|приобретение авто...|
|       0| -5623.422610230956|       33|            Среднее|           1| женат / замужем|               0|     M|  сотрудник|   0|145885.95

__2. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Например, отрицательное количество дней трудового стажа в столбце days_employed. Для реальных данных это нормально. Обработайте значения в столбцах с аномалиями и опишите возможные причины появления таких данных. После обработки аномалий заполните пропуски в days_employed медианными значениями по этому столбцу.__

Отрицательные значения в days_employed могут быть вызваны, например, ошибкой ввода.

Отрицательные значения приведем к абсолютным, а нулевые заполним медианными


In [66]:
df = df.withColumn("days_employed", abs(col("days_employed")))
df = df.fillna({"days_employed": df.select(median("days_employed")).first()[0]})
df.show()


+--------+------------------+---------+-------------------+------------+----------------+----------------+------+-----------+----+------------------+--------------------+
|children|     days_employed|dob_years|          education|education_id|   family_status|family_status_id|gender|income_type|debt|      total_income|             purpose|
+--------+------------------+---------+-------------------+------------+----------------+----------------+------+-----------+----+------------------+--------------------+
|       1| 8437.673027760233|       42|             высшее|           0| женат / замужем|               0|     F|  сотрудник|   0| 253875.6394525987|       покупка жилья|
|       1| 4024.803753850451|       36|            среднее|           1| женат / замужем|               0|     F|  сотрудник|   0|112080.01410244203|приобретение авто...|
|       0| 5623.422610230956|       33|            Среднее|           1| женат / замужем|               0|     M|  сотрудник|   0|145885.95229686

__3. Замените вещественный тип данных в столбце total_income на целочисленный.__

In [67]:
df = df.withColumn("total_income", col("total_income").cast("integer"))
df.show()

+--------+------------------+---------+-------------------+------------+----------------+----------------+------+-----------+----+------------+--------------------+
|children|     days_employed|dob_years|          education|education_id|   family_status|family_status_id|gender|income_type|debt|total_income|             purpose|
+--------+------------------+---------+-------------------+------------+----------------+----------------+------+-----------+----+------------+--------------------+
|       1| 8437.673027760233|       42|             высшее|           0| женат / замужем|               0|     F|  сотрудник|   0|      253875|       покупка жилья|
|       1| 4024.803753850451|       36|            среднее|           1| женат / замужем|               0|     F|  сотрудник|   0|      112080|приобретение авто...|
|       0| 5623.422610230956|       33|            Среднее|           1| женат / замужем|               0|     M|  сотрудник|   0|      145885|       покупка жилья|
|       3|

__4. Если в данных присутствуют строки-дубликаты, удалите их. Также обработайте неявные дубликаты. Например, в столбце education есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите такие значения к одному регистру.__
<br>После удаления дубликатов сделайте следующее:
- поясните, как выбирали метод для поиска и удаления дубликатов в данных;
- приведите возможные причины появления дубликатов.

Для удаления дубликатов можно сначала привести их к нижнему регистру, а затем удалить.

Дубликаты могли появится по ошибке: если их ввели несколько раз.

In [68]:
df = df.withColumn("education", lower(df["education"]))
df = df.dropDuplicates()
df.show()

+--------+------------------+---------+---------+------------+--------------------+----------------+------+-----------+----+------------+--------------------+
|children|     days_employed|dob_years|education|education_id|       family_status|family_status_id|gender|income_type|debt|total_income|             purpose|
+--------+------------------+---------+---------+------------+--------------------+----------------+------+-----------+----+------------+--------------------+
|       0| 2194.220566878695|       21|  среднее|           1|Не женат / не зам...|               4|     M|  компаньон|   0|      145017|операции с коммер...|
|       0| 191.1467457844417|       64|   высшее|           0|    гражданский брак|               1|     F|  компаньон|   0|      186408|             свадьба|
|       0| 563.9370284307207|       33|  среднее|           1|     женат / замужем|               0|     M|  компаньон|   0|      161976|на покупку своего...|
|       2|1653.2725398991834|       25|   высш

__5. Создайте два новых датафрейма, в которых:__
- каждому уникальному значению из education соответствует уникальное значение education_id — в первом;
- каждому уникальному значению из family_status соответствует уникальное значение family_status_id — во втором.
Удалите из исходного датафрейма столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id. Новые датафреймы — это те самые «словари» (не путайте с одноимённой структурой данных в Python), к которым вы сможете обращаться по идентификатору.

In [69]:
education = df.select("education").distinct().withColumn("education_id", monotonically_increasing_id())
family_status = df.select("family_status").distinct().withColumn("family_status_id", monotonically_increasing_id())
df = df.drop("education", "family_status")

print("Education:")
education.show()

print("Family Status:")
family_status.show()

print("Updated DataFrame:")
df.show()

Education:
+-------------------+------------+
|          education|education_id|
+-------------------+------------+
|          начальное|           0|
|неоконченное высшее|           1|
|             высшее|           2|
|            среднее|           3|
|     ученая степень|           4|
+-------------------+------------+

Family Status:
+--------------------+----------------+
|       family_status|family_status_id|
+--------------------+----------------+
|    гражданский брак|               0|
|      вдовец / вдова|               1|
|Не женат / не зам...|               2|
|           в разводе|               3|
|     женат / замужем|               4|
+--------------------+----------------+

Updated DataFrame:
+--------+------------------+---------+------------+----------------+------+-----------+----+------------+--------------------+
|children|     days_employed|dob_years|education_id|family_status_id|gender|income_type|debt|total_income|             purpose|
+--------+------------

__6. На основании диапазонов, указанных ниже, создайте столбец `total_income_category` с категориями:__
- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

In [70]:
df = df.withColumn("total_income_category", 
                   when(df["total_income"] <= 30000, 'E')
                   .when(df["total_income"] <= 50000, 'D')
                   .when(df["total_income"] <= 200000, 'C')
                   .when(df["total_income"] <= 1000000, 'B')
                   .otherwise('A'))
df.show()

+--------+------------------+---------+------------+----------------+------+-----------+----+------------+--------------------+---------------------+
|children|     days_employed|dob_years|education_id|family_status_id|gender|income_type|debt|total_income|             purpose|total_income_category|
+--------+------------------+---------+------------+----------------+------+-----------+----+------------+--------------------+---------------------+
|       0| 2194.220566878695|       21|           1|               4|     M|  компаньон|   0|      145017|операции с коммер...|                    C|
|       0| 191.1467457844417|       64|           0|               1|     F|  компаньон|   0|      186408|             свадьба|                    C|
|       0| 563.9370284307207|       33|           1|               0|     M|  компаньон|   0|      161976|на покупку своего...|                    C|
|       2|1653.2725398991834|       25|           0|               0|     F|  компаньон|   0|      1

__7. Создайте функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:__
- 'операции с автомобилем',
- 'операции с недвижимостью',
- 'проведение свадьбы',
- 'получение образования'.

<br>Например, если в столбце purpose находится подстрока 'на покупку автомобиля', то в столбце purpose_category должна появиться строка 'операции с автомобилем'.
Вы можете использовать собственную функцию и метод apply(). Изучите данные в столбце purpose и определите, какие подстроки помогут вам правильно определить категорию.

In [71]:
unique = df.select("purpose").distinct().collect()
print('Уникальные значения в столбце `purpose`')
for row in unique:
    print(f'- {row["purpose"]}')

Уникальные значения в столбце `purpose`
- свадьба
- сделка с автомобилем
- дополнительное образование
- приобретение автомобиля
- высшее образование
- строительство жилой недвижимости
- автомобили
- на проведение свадьбы
- на покупку подержанного автомобиля
- покупка жилой недвижимости
- покупка коммерческой недвижимости
- операции с коммерческой недвижимостью
- недвижимость
- сделка с подержанным автомобилем
- на покупку автомобиля
- покупка недвижимости
- получение дополнительного образования
- получение высшего образования
- ремонт жилью
- покупка жилья
- покупка своего жилья
- заняться высшим образованием
- свой автомобиль
- заняться образованием
- покупка жилья для сдачи
- строительство собственной недвижимости
- образование
- операции с недвижимостью
- операции с жильем
- на покупку своего автомобиля
- строительство недвижимости
- сыграть свадьбу
- профильное образование
- автомобиль
- жилье
- покупка жилья для семьи
- операции со своей недвижимостью
- получение образования


In [72]:
def cat_purpose(purpose):
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    elif 'недвижимост' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образован' in purpose:
        return 'получение образования'
    else:
        return 'другое'

category_purpose = udf(cat_purpose, StringType())

df = df.withColumn("purpose_category", category_purpose(col("purpose")))
df.show()

+--------+------------------+---------+------------+----------------+------+-----------+----+------------+--------------------+---------------------+--------------------+
|children|     days_employed|dob_years|education_id|family_status_id|gender|income_type|debt|total_income|             purpose|total_income_category|    purpose_category|
+--------+------------------+---------+------------+----------------+------+-----------+----+------------+--------------------+---------------------+--------------------+
|       0| 2194.220566878695|       21|           1|               4|     M|  компаньон|   0|      145017|операции с коммер...|                    C|операции с недвиж...|
|       0| 191.1467457844417|       64|           0|               1|     F|  компаньон|   0|      186408|             свадьба|                    C|  проведение свадьбы|
|       0| 563.9370284307207|       33|           1|               0|     M|  компаньон|   0|      161976|на покупку своего...|                  

__Шаг 3. Ответьте на вопросы__

- Есть ли зависимость между количеством детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

In [73]:
pdf = df.toPandas()
print("Корреляция с 'debt':")
pdf[['children', 'family_status_id','total_income', 'debt']].corr()[['debt']].drop(index='debt')


Корреляция с 'debt':


,debt
children,0.018256
family_status_id,0.020347
total_income,-0.011797


Зависимость между семейным положением, количеством детей, уровнем дохода и возвратом кредита в срок близки к нулю поэтому можно скзаать, что зависимость незначительна

In [74]:
cross = df.crosstab('purpose_category', 'debt').collect()
for i in cross:
    print(f"Цель взятия кредита: {i['purpose_category_debt']} - доля задолженностей: {int(i[2])/((int(i[1])+int(i[2])))} ")

Цель взятия кредита: операции с автомобилем - доля задолженностей: 0.09359033906177427 
Цель взятия кредита: проведение свадьбы - доля задолженностей: 0.08003442340791739 
Цель взятия кредита: операции с недвижимостью - доля задолженностей: 0.0723337341596522 
Цель взятия кредита: получение образования - доля задолженностей: 0.0922003488661849 


Доля задолженностей менее 10%. Можно увидеть, что чаще всего бывают задолжности с кредитом на автомобиль и на образование 

__Сохранение обработанного датафрейма__

In [75]:
import pandas as pd

df_pandas = df.toPandas()
df_pandas

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,0,2194.220567,21,1,4,M,компаньон,0,145017,операции с коммерческой недвижимостью,C,операции с недвижимостью
1,0,191.146746,64,0,1,F,компаньон,0,186408,свадьба,C,проведение свадьбы
2,0,563.937028,33,1,0,M,компаньон,0,161976,на покупку своего автомобиля,C,операции с автомобилем
3,2,1653.272540,25,0,0,F,компаньон,0,102955,автомобиль,C,операции с автомобилем
4,0,253.806355,23,1,1,M,сотрудник,0,64992,на проведение свадьбы,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,2,2529.633034,29,1,0,F,компаньон,0,385531,образование,B,получение образования
21450,0,3150.622828,46,1,2,F,компаньон,0,123090,недвижимость,C,операции с недвижимостью
21451,0,3596.155013,33,0,0,M,компаньон,0,168698,получение дополнительного образования,C,получение образования
21452,0,392388.182360,59,0,0,M,пенсионер,1,226496,образование,B,получение образования


In [78]:
df_pandas.to_csv('result.csv', index=False)